# Preprocessing gait detection
This script preprocesses the sensor data, tabulates it into windows, and generates features. 

Execution time $\approx$ 1 minute per participant (amounting to 45 minutes in total).

### Modules

In [5]:
from pdathome.constants import  descriptives, participant_ids
from pdathome.preprocessing import preprocess_gait

### Process data

In [7]:
for subject in participant_ids.L_PD_IDS + participant_ids.L_HC_IDS:
    for side in [descriptives.MOST_AFFECTED_SIDE, descriptives.LEAST_AFFECTED_SIDE]:
        preprocess_gait(subject, side)